In [2]:
import sys

sys.path.append('..')

In [10]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt

# no line wrap
pd.set_option('display.expand_frame_repr', False)

data = 'human'
channels = '*'
limit = 10000
batch_size = 64
lrs = [1e-3, 5e-4, 1e-4]
weight_decays = [0, 1e-4]
epochs = 20

# Define the directory and file pattern
# name = f'{model_name}_{args.data}_{args.channels}_{args.limit}_{args.num_moves}_{args.engine_prob}_{args.batch_size}_{args.lr}_{args.weight_decay}_{args.epochs}.csv'
# example = ConvLSTM_synthetic_12_10000_60_0.5_64_0.001_0.0001_20.csv
# pattern = os.path.join('../results', '*_processed_6_10000_40_*_64_*_*.csv')
pattern = os.path.join('../results', f'*_{data}_{channels}*.csv')
files = glob.glob(pattern)

results = {}


for file in files:
  data = file.split('/')[-1].split('_')
  model = data[0]
  lr = float(data[-3])
  weight_decay = float(data[-2])
  channels = int(data[2])
  df = pd.read_csv(file)
  df = df[df['type'] == 'test']
  row = df[df['accuracy'] == df['accuracy'].max()].iloc[0].to_dict()
  del row['epoch']
  del row['loss']
  del row['type']
  row['lr'] = lr
  row['wd'] = weight_decay
  row['channels'] = int(channels)
  if model not in results:
    results[model] = row
  else:
    if row['accuracy'] > results[model]['accuracy']:
      results[model] = row

df = pd.DataFrame(results).T
df = df.sort_values('accuracy', ascending=False)
print(df)

                           accuracy  precision    recall        f1       auc       lr      wd  channels
Transformer                0.509091   0.498529  0.509091  0.503648  0.555413  0.00100  0.0001      12.0
ConvLSTM(evals, times)     0.290909   0.400930  0.290909  0.334016  0.392499  0.00010  0.0001      12.0
ConvLSTM(evals)            0.290909   0.457690  0.290909  0.344895  0.413747  0.00005  0.0000      12.0
ConvLSTM                   0.272727   0.498363  0.272727  0.301212  0.488144  0.00010  0.0000      12.0
Transformer(evals)         0.272727   0.355080  0.272727  0.294294  0.457511  0.00100  0.0000      12.0
Transformer(evals, times)  0.254545   0.518831  0.254545  0.268943  0.434567  0.00100  0.0001      12.0
Transformer(times)         0.218182   0.468398  0.218182  0.214073  0.443276  0.00005  0.0000      12.0
ConvLSTM(times)            0.218182   0.301042  0.218182  0.220911  0.398997  0.00005  0.0001      12.0
